# IMPORTAÇÕES

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from copy import copy, deepcopy
from functools import reduce
from typing import List, Any

# FUNÇÕES

In [2]:
class Environment:
  def __init__(self, root, rel_path):
    self.root = root            # Pasta raiz
    self.rel_path = rel_path    # Caminho relativo

  def get_file_path(self):
    """
    Retorna caminho do arquivo.
    :return: Caminho do arquivo.
    """
    parts = [self.root]
    parts.extend(self.rel_path)
    return "/".join(parts)

  def clone(self, mode="shallow"):
    """
    Retorna uma cópia do objeto. O padrão é shallow copy.
    :param mode: Modo de cópia ("shallow" ou "deep").
    :return: Cópia do objeto.
    """
    if mode == "deep":
      return deepcopy(self)
    return copy(self)

def df_show_null(df: pd.DataFrame) -> None:
  """
  Exibe o número de valores nulos por coluna.

  :param df: DataFrame a ter os domínios exibidos.
  :return: None
  """
  print("# Número de valores nulos por coluna:")
  display(df.isnull().sum())

def df_filter_dom(df: pd.DataFrame, columns: List[str], doms: List[Any]) -> pd.DataFrame:
  """
  Filtra valores das colunas que estejam fora do respectivo domínio. Se
  o domínio for uma lista, considera os valores individualmente. Se for
  uma tupla, considera um intervalo de valores (exclusivo para tipo numérico).

  :param df: DataFrame a ser filtrado.
  :param columns: Lista de colunas a serem filtradas.
  :param doms: Lista de domínios correspondentes.
  :return: cópia do DataFrame filtrada.
  """
  df = df.copy()

  for col, dom in zip(columns, doms):
    if type(dom) == list:
      df = df[df[col].isin(dom)]

    elif type(dom) == tuple:
      df = df[(dom[0] <= df[col]) & (df[col] <= dom[1])]

  return df

def plot_na(df: pd.DataFrame, columns: List[str], aggregator: str):
  """
  Plota a quantidade de NaNs por colunas com base no agregador.

  Parâmetros:
  :param df: DataFrame contendo os dados.
  :param columns: colunas a terem seus NaN contados.
  :param aggregator: coluna a ser usada para agrupar os dados.
  """
  df = df.copy()

  # Conta os NaNs com base no agregador
  nan_counts = df.groupby(aggregator)[columns].apply(lambda x: x.isna().sum())
  nan_counts = nan_counts.drop(columns=aggregator, errors="ignore")  # Remove a própria coluna agregadora

  # Exibe a tabela com contagem de NaNs
  print("Contagem de NaNs:")
  print(nan_counts)

# SETUP DO AMBIENTE

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# OBTENÇÃO DOS DATASETS

In [4]:
# PASTA RAIZ
ROOT = "/content/drive/MyDrive/07_per_shared/projCDat_25_1/datasets/cooked"

# DICIONÁRIOS AUXILIARES:
dict_dfs = dict()   # Fonte -> DataFrame
dict_envs = dict()  # Fonte -> Ambiente
dict_rel_paths = {  # Fonte -> Caminho relativo
    "MapBiomas": ["MapBiomas", "MapBiomas_merged", "area_queimada_total_merged.csv"],
    "SEEG": ["SEEG", "emissoes_c02_cooked.csv"],
    "INMET": ["INMET", "inmet_merged.csv"],
    "TerraBrasilis": ["TerraBrasilis", "qtd_focos", "historico_estado_all_merged.csv"],
}

# Carrega ambientes e DataFrames:
for src, rel_path in dict_rel_paths.items():
  dict_envs[src] = Environment(ROOT, rel_path)
  dict_dfs[src] = pd.read_csv(dict_envs[src].get_file_path())

  print(f"\nDataset {src} carregado.")
  display(dict_dfs[src].head())


Dataset MapBiomas carregado.


,_ano,_mes,_estado,que_area_queimada
0,1985,1,TO,174.0
1,1985,2,TO,1387.0
2,1985,3,TO,3634.0
3,1985,4,TO,3995.0
4,1985,5,TO,25561.0



Dataset SEEG carregado.


,_ano,_estado,_mes,car_c02_emitido
0,1990,AC,1,2.130896e+07
1,1990,AC,2,2.130896e+07
2,1990,AC,3,2.130896e+07
3,1990,AC,4,2.130896e+07
4,1990,AC,5,2.130896e+07



Dataset INMET carregado.


,_estado,_ano,_mes,cli_pressao_atm_med,cli_temp_ar_med,cli_temp_orvalho_med,cli_umid_rel_med,cli_umid_rel_min_med,cli_umid_rel_min_max,cli_umid_rel_min_min,cli_veloc_vento_max,cli_veloc_vento_med
0,AM,2000,5,1004.492026,26.237716,23.545690,85.922414,82.957845,98.0,58.0,3.7,0.924026
1,AM,2000,6,1004.887222,26.419861,23.185417,83.377778,80.301255,98.0,52.0,3.5,1.024339
2,AM,2000,7,1005.176043,25.800000,22.558205,82.476447,79.559892,98.0,47.0,3.7,1.107962
3,AM,2000,8,1004.761559,26.854973,22.629973,79.155914,76.044355,97.0,45.0,3.4,1.060027
4,AM,2000,9,1003.505621,27.208580,22.730325,78.076923,75.084444,97.0,43.0,3.7,1.154748



Dataset TerraBrasilis carregado.


,_ano,_estado,_mes,que_focos_qtd
0,1998,AC,1,9.9375
1,1999,AC,1,9.9375
2,2000,AC,1,9.9375
3,2001,AC,1,9.9375
4,2002,AC,1,9.9375


# JUNÇÃO DOS DATASETS

In [5]:
# PARÂMETROS:
JOIN_ON = ["_ano", "_mes", "_estado"] # Colunas a partir das quais é feita a junção
JOIN_TYPE = "outer"                   # "outer" preserva todas as linhas, mesmo que não haja correspondência

# Realiza junção de todos os DataFrames com base nos parâmetros acima
df_all = reduce(lambda left, right: pd.merge(left, right, on=JOIN_ON, how=JOIN_TYPE), dict_dfs.values())

# Reordena colunas alfabeticamente
df_all = df_all[sorted(df_all.columns)]

display(df_all)
df_show_null(df_all)

,_ano,_estado,_mes,car_c02_emitido,cli_pressao_atm_med,cli_temp_ar_med,cli_temp_orvalho_med,cli_umid_rel_med,cli_umid_rel_min_max,cli_umid_rel_min_med,cli_umid_rel_min_min,cli_veloc_vento_max,cli_veloc_vento_med,que_area_queimada,que_focos_qtd
0,1985,AC,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.00000,NaN
1,1985,AM,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1177.00000,NaN
2,1985,AP,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13584.62963,NaN
3,1985,MA,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3159.00000,NaN
4,1985,MT,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3645.00000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11767,2025,MT,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.590909
11768,2025,PA,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.590909
11769,2025,RO,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.590909
11770,2025,RR,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.590909


# Número de valores nulos por coluna:


,0
_ano,0
_estado,0
_mes,0
car_c02_emitido,756
cli_pressao_atm_med,10747
cli_temp_ar_med,10747
cli_temp_orvalho_med,10747
cli_umid_rel_med,10747
cli_umid_rel_min_max,10747
cli_umid_rel_min_med,10747


# DEFINIÇÃO DE ESCOPO

In [6]:
# DEFINIÇÃO DO ESCOPO POR ATRIBUTO
SELECTORS = ["_estado", "_ano"]
SCOPES = [["AC", "AM", "AP", "MA", "MT", "PA", "RO", "RR", "TO"], (2000, 2023)]

# Filtra valores fora do escopo de análise
df_filtered = df_filter_dom(df_all, SELECTORS, SCOPES) # Remove linhas fora do escopo
df_filtered = df_filtered.dropna() # Remove nulos surgidos na junção
df_filtered = df_filtered.sort_values(by=["_estado", "_ano", "_mes"]).reset_index(drop=True) # Ordena e reindexa

display(df_filtered)
df_show_null(df_filtered)

,_ano,_estado,_mes,car_c02_emitido,cli_pressao_atm_med,cli_temp_ar_med,cli_temp_orvalho_med,cli_umid_rel_med,cli_umid_rel_min_max,cli_umid_rel_min_med,cli_umid_rel_min_min,cli_veloc_vento_max,cli_veloc_vento_med,que_area_queimada,que_focos_qtd
0,2008,AC,7,2.627698e+07,986.843612,28.142731,18.914978,59.555066,95.0,54.432558,29.0,5.1,2.152915,4957.0,165.0
1,2008,AC,9,2.627698e+07,991.705941,24.446194,19.467987,75.811881,97.0,72.496700,25.0,1.0,0.210504,46073.0,2947.0
2,2008,AC,10,2.627698e+07,990.328360,25.229298,21.617473,81.870968,96.0,78.720430,29.0,1.0,0.204959,30355.0,856.0
3,2008,AC,11,2.627698e+07,988.610987,25.195410,22.624478,86.905424,96.0,84.048679,42.0,1.0,0.186970,2082.0,63.0
4,2008,AC,12,2.627698e+07,988.692608,24.898790,22.727554,88.529570,96.0,86.116935,53.0,1.0,0.179442,127.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,2023,TO,8,8.870288e+07,981.811559,28.596371,14.201750,45.434140,83.0,42.395161,16.0,5.0,1.562108,390184.0,1388.0
1021,2023,TO,9,8.870288e+07,980.431453,30.121016,15.091537,44.162200,95.0,40.932299,16.0,5.0,1.516053,474588.0,3075.0
1022,2023,TO,10,8.870288e+07,979.404203,29.140719,19.484029,59.382734,96.0,55.821583,17.0,4.8,1.443768,169952.0,1675.0
1023,2023,TO,11,8.870288e+07,979.489956,28.211340,20.277909,64.777614,99.0,61.192931,25.0,4.6,1.335725,63394.0,160.0


# Número de valores nulos por coluna:


,0
_ano,0
_estado,0
_mes,0
car_c02_emitido,0
cli_pressao_atm_med,0
cli_temp_ar_med,0
cli_temp_orvalho_med,0
cli_umid_rel_med,0
cli_umid_rel_min_max,0
cli_umid_rel_min_med,0


# VERIFICAÇÃO DE DADOS INDISPONÍVEIS REMANESCENTES

In [7]:
# Exibe número de valores indisponíveis com base nas entradas da coluna agregadora
AGGREGATOR = "_ano"
plot_na(df_filtered, df_filtered.columns, AGGREGATOR)

Contagem de NaNs:
      _estado  _mes  car_c02_emitido  cli_pressao_atm_med  cli_temp_ar_med  \
_ano                                                                         
2000        0     0                0                    0                0   
2001        0     0                0                    0                0   
2003        0     0                0                    0                0   
2004        0     0                0                    0                0   
2005        0     0                0                    0                0   
2006        0     0                0                    0                0   
2007        0     0                0                    0                0   
2008        0     0                0                    0                0   
2009        0     0                0                    0                0   
2010        0     0                0                    0                0   
2011        0     0                0          

# SALVAMENTO

In [8]:
# Ambiente de saída de dados
SAVE_TO = "_all"             # Nome da pasta a partir da raiz em que salvar o arquivo
FOUT_NAME = "all_merged.csv" # Nome do arquivo de saída
env_out = Environment(ROOT, [SAVE_TO, FOUT_NAME])

# Salva arquivo
df_filtered.to_csv(env_out.get_file_path(), index=False) # Não preserva o índice